# 서브워드 알고리즘 중 BPE 등 많은 변형 알고리즘을 내장
# 그리고 센텐스피스는 사전토큰화 작업없이 가능. 한글에 유리. 어차피 모든 문장의 단어를 서브토큰 하니까
# 이 예제는 단지 서브토큰로 나누어 결과만 보는 예제 임

In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.chdir(r"c:\projects\PY_MLE")

import pandas as pd
import numpy as np
import re
from tqdm import tqdm
from gensim.models.doc2vec import TaggedDocument

from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling1D, SimpleRNN, Embedding
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
import sentencepiece as spm
import urllib.request
import csv

In [5]:
train_df = pd.read_csv('IMDb_Reviews.csv')
print(train_df[:5])
print(len(train_df))

                                              review  sentiment
0  My family and I normally do not watch local mo...          1
1  Believe it or not, this was at one time the wo...          0
2  After some internet surfing, I found the "Home...          0
3  One of the most unheralded great works of anim...          1
4  It was the Sixties, and anyone with long hair ...          0
50000


In [6]:
# 센텐스피스 입력으로 사용하기 위해서는 텍스트파일이 필요
with open('imdb_review.txt','w',encoding='utf8') as f:
    f.write('\n'.join(train_df['review']))

In [8]:
# 각 단어집합에 고유의 정수를 부여하는 과정
spm.SentencePieceTrainer.Train('--input=imdb_review.txt --model_prefix=imdb --vocab_size=5000 --model_type=bpe --max_sentence_length=9999')

In [10]:
vocab_list = pd.read_csv('imdb.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE)
vocab_list[:10]

,0,1
0,<unk>,0
1,<s>,0
2,</s>,0
3,▁t,0
4,▁a,-1
5,he,-2
6,in,-3
7,▁the,-4
8,▁s,-5
9,re,-6


In [ ]:
# 여기서부터는 테스트
sp = spm.SentencePieceProcessor()
vocab_file = "imdb.model"
sp.Load(vocab_file)

True

In [ ]:
lines = [
"I didn't at all think of it this way.",
"I have waited a long time for someone to film"
]

for line in lines:
    print(line)
    print(sp.EncodeAsPieces(line)) # 문장을 인코딩
    print(sp.EncodeAsIds(line))    # 정수부여 조회
    print()
    
print()
print()

# 이렇게 해도 되는 듯(이게 나을 듯 하네.)
for line in lines:
    print(line)
    print(sp.Encode(line, out_type=str))
    print(sp.Encode(line, out_type=int))
    print()

I didn't at all think of it this way.
['▁I', '▁didn', "'", 't', '▁at', '▁all', '▁think', '▁of', '▁it', '▁this', '▁way', '.']
[41, 624, 4950, 4926, 139, 170, 378, 30, 58, 73, 413, 4945]

I have waited a long time for someone to film
['▁I', '▁have', '▁wa', 'ited', '▁a', '▁long', '▁time', '▁for', '▁someone', '▁to', '▁film']
[41, 142, 1364, 1121, 4, 668, 285, 93, 1079, 33, 91]



I didn't at all think of it this way.
['▁I', '▁didn', "'", 't', '▁at', '▁all', '▁think', '▁of', '▁it', '▁this', '▁way', '.']
[41, 624, 4950, 4926, 139, 170, 378, 30, 58, 73, 413, 4945]

I have waited a long time for someone to film
['▁I', '▁have', '▁wa', 'ited', '▁a', '▁long', '▁time', '▁for', '▁someone', '▁to', '▁film']
[41, 142, 1364, 1121, 4, 668, 285, 93, 1079, 33, 91]

